In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm # Anzeigen des aktuellen Ladestandes
import PyPDF2
import re
from os import listdir
from os.path import isfile, join
import locale #für das deutsche Zeitformat

## 1. URL von allen Hauptseiten holen

In [2]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [3]:
pd.DataFrame(alle_seiten)
df_a=pd.DataFrame(alle_seiten)
# Die Geschäftsnummer zur Zahl machen. 
df_a["VorlageNR"] = df_a["VorlageNR"].astype(int) 

## 1.1 Selektion: Fokus auf Vorstösse
So will ich eine allzu grosse pdf-Schwemme verhindern.
Die Idee: nur jene Geschäfte auf den PC ziehen, die auch relevant sind für die Auswertung. 
Also nur Vorstösse. Mich so bis zu den PDF's vorkämpfen, um auch hier nur jene auf meinem Rechner zu haben, 
die ich wirklich auf meinem Rechner will.

In [4]:
#hier mache ich eine Liste, die nur aus denjenigen URLs besteht, die Vorstösse sind. 

list_of_vorstoss=["Motion","Interpellation","Postulat","Kleine Anfrage", "Initiative"]
df_v=df_a[df_a["Art des Geschäfts"].isin (list_of_vorstoss)]

In [5]:
df_v #hier habe ich nun alle Vorstösse

,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,3031,/gast/geschaefte/2018,Interpellation der SVP-Fraktion betreffend Sic...,Einreichung,Direktion für Bildung und Kultur,Interpellation,hängig
1,3030,/gast/geschaefte/2017,Motion der SVP-Fraktion betreffend Stärkung de...,Einreichung,,Motion,hängig
2,3029,/gast/geschaefte/2016,Interpellation der Fraktion Alternative - die ...,Einreichung,Volkswirtschaftsdirektion,Interpellation,hängig
5,3026,/gast/geschaefte/2013,Interpellation von Markus Spörri und Thomas Ga...,Einreichung,Baudirektion,Interpellation,hängig
7,3024,/gast/geschaefte/2012,Interpellation von Claus Soltermann und Heinz ...,Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig
...,...,...,...,...,...,...,...
1957,762,/gast/geschaefte/1331,Motion der erweiterten Justizprüfungskommissio...,Datenmigration,,Motion,abgeschlossen
1963,666,/gast/geschaefte/1325,Motion der Kommission Teilrevision Personalges...,Datenmigration,,Motion,abgeschlossen
1979,304,/gast/geschaefte/1309,Motion von Manuela Weichelt betreffend HIV-Prä...,Datenmigration,,Motion,abgeschlossen
1980,282,/gast/geschaefte/1712,Motion von Christoph Hohler betreffend Radstre...,Frist erstreckt,Baudirektion,Motion,hängig


In [7]:
df_v=df_v.reset_index()#Ich will überall nun denselben Index. deshalb nur soviel, wie ich Zeilen habe
df_v.to_csv("KRFrames/df_v_Hauptseiten")

### 1.1.1 Eine Liste aus allen Geschäfts-URLs machen (um später auf diese zuzugreiffen)

In [8]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_v["url"]:
    url_liste_ganz.append(url_anfang_original+element)
#diese werden zusammengefügt aus dem Anfang der Adresse sowie dem Geschäfts-URL

In [9]:
len(url_liste_ganz)

1227

### Die einzelnen Geschäfte aufrufen und Infos rausholen

# -> Einschub: ich will speichern und mit diesen Daten dann arbeiten.

In [11]:
#habe nun alle Seiten der Geschäfte auf meinem Rechner gespeichert. 
number = 0
for page in url_liste_ganz:
    page_content = requests.get(page)
    page_content = page_content.text
    with open("KRhtml/Geschaeft"+str(number)+".html", "w") as file:
        file.write(page_content)
        file.close()
    number +=1
    #Es sind nun exakt so viele Seiten auf meinem Rechner, wie das DataFrame oben Zeilen anzeigt

# <-Einschub-Ende. Ich habe es geschafft, das zu speichern. 

In [17]:
#nun muss ich den Code ändern, sodass ich nicht mehr online zugreiffe sondern über meinen Ordner.
#hier nun die Liste der Zugriffe.
geschaefts_liste=[]

for seite_g in range(0,1226):
    gesch= "KRhtml/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [18]:
len(geschaefts_liste)

1226

In [19]:
geschaefts_liste[-10:]

['KRhtml/Geschaeft1216.html',
 'KRhtml/Geschaeft1217.html',
 'KRhtml/Geschaeft1218.html',
 'KRhtml/Geschaeft1219.html',
 'KRhtml/Geschaeft1220.html',
 'KRhtml/Geschaeft1221.html',
 'KRhtml/Geschaeft1222.html',
 'KRhtml/Geschaeft1223.html',
 'KRhtml/Geschaeft1224.html',
 'KRhtml/Geschaeft1225.html']

# 2. Zugang zu den PDFs

Ich musste rausfinden, bei welchen Files ich nicht auf die Nummer 1 zugreiffen kann. 
*Siehe File: "Falsche Files rausfinden - code anpassen"* 

In [20]:
#hier  habe ich mich entschieden, auf den Zugriff der Namen zu verzichten, und dies einheitlich über die PDFs zu lösen.
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
eingang_list=[]


for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument.
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
   
   
     #Da es zum Teil Referenzen hat und dadurch ein anderer Ort abgegriffen werden muss, muss ich varieren.
    
    if td_g_list[-1].find_all("a")[0].text== "1": #Ich brauche immer das Dokument 1. 
        url_v_pdf = td_g_list[-1].find("a")["href"]
    
    elif td_g_list[-1].find_all("a")[0].text== "2": #bei älteren GEschäften sind es Bericht&Anträge
        url_v_pdf = td_g_list[-1].find("a")["href"]   
    else: 
        td_g_list[-1].find_all("a")[0].text== "3"
        url_v_pdf = td_g_list[-1].find("a")["href"]
   
            
    
    minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf,}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

In [21]:
df_g=pd.DataFrame(eingang_list)

In [22]:
#Das Datum muss noch als solches lesbar gemacht werden
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum
df_g["VorlageNR"] =df_g["VorlageNR"].astype(int)

In [23]:
df_g.to_csv("KRFrames/df_g_LinkszuPDF")

In [22]:
df_g=pd.read_csv("KRFrames/df_g_LinkszuPDF")

# 3. PDFs der Vorstösse runterladen

In [25]:
for link in tqdm(df_g["Link Vorstoss-PDF"]):
    r = requests.get("https://kr-geschaefte.zug.ch"+link, stream =True) #Das Stream braucht es, um mit dem Zip umgehen zu koennen
    name = link.split("/")[-1] #Der Computer kann mit "/" in einem Namen nicht umgehen, deshalb nehme ich die hier raus, und nehme nur den letzten Teil des NAmens.
    with open("KRGeschPDF/"+name, "wb") as f:
        f.write(r.content)

100%|██████████| 1226/1226 [03:16<00:00,  6.24it/s]


# 4. PDFs sind auf dem Rechner, nun lesbar machen und !Regex Baby!

In [2]:
#nun will ich aus jedem PDF zum Einen die VorlagenNR rausziehen sowie Namen und Datum hier die Definition
# ! Danke dem |$ am Schluss gibt es keinen Index-Error, wenn er kein Resultat findet.
regex_vorlage= r"VORLAGE.NR\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_datum = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [3]:
#Lösung online gefunden um aus dem PDF-Ordner eine Liste mit den Links herzustellen.
pdf_files_list= [f for f in listdir("KRGeschPDF") if isfile(join("KRGeschPDF", f))]

In [4]:
len(pdf_files_list) #Ich habe ja die PDFs ignoriert, zu welchen ich auf Grund der Referenz keine PDF-Links hatte.

1099

In [5]:
# !!!!!! ACHTUNG: zuerst mit einzelen testen. Möglicherweise findet er die Files nicht. 
dat_pdf_list=[]
for dok in pdf_files_list:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_name = dok
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_ein= re.findall(regex_datum, front, re.IGNORECASE)[0] #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    
    minidict_pdf_dat={"Einreichedatum": dat_p_ein, "VorlageNR":vorl_p_nr, "Filename":pdf_name}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [6]:
df_p=pd.DataFrame(dat_pdf_list)

In [7]:
df_p.sort_values("Einreichedatum") #Stichproben haben gezeigt, dass die fehlerhaften Datumsangaben in GEschäftern sind,
# Welche selber meist schon ein Datum haben. 

,Einreichedatum,VorlageNR,Filename
670,,,2930-1-16005_Sicherheit-Schulweg.pdf
640,,,2867-1-15770_Einsatzkoordination.pdf
647,,,2884-1-15816_Zahlungen-Kantone.pdf
271,,,14235_2216_1_Krankenkassenpr%C3%A4mien.pdf
649,,,2892-1-15881_KA-Sportf%C3%B6rderung.pdf
...,...,...,...
763,9. MÄRZ 2006,1420,pdoc_1219_1.pdf
827,9. MÄRZ 2007,1518,pdoc_1682_1.pdf
322,9. November 2013,,14510_2318_1_Weiterbildungen.pdf
164,9. September 2010,,13532_1967_1_RGPK.pdf


In [8]:
df_p.to_csv("KRFrames/INFOSausPDF")

In [ ]:
df_p=pd.read_csv("KRFrames/INFOSausPDF")
df_p

## 4.1 Zeit in Datetime umwandeln
schwierigkeit: wie bringe ich das deutsche Zeitformat in englisches. Mache ich es falsch oder gibts wieder probleme mit den Grossbuchstaben der Zeitformate? Hatten wir windowsuser auch schon.

In [ ]:
df_p=pd.read_csv("KRFrames/INFOSausPDF")

In [10]:
locale.setlocale(locale.LC_ALL, 'de_DE') # hier die Sprachwahl
df_p["Einreichedatum"] = pd.to_datetime(df_p["Einreichedatum"], format= "%d. %B %Y", errors='coerce') #Angabe des Formats

In [11]:
 df_p #Halleluia:-) habs doch noch geschafft. Und die wenigen, die falsch formiert sind, werden nicht auffallen.

,Einreichedatum,VorlageNR,Filename
0,2007-09-17,1584,12487_1584.1_motion.pdf
1,2007-09-18,1589,12493_1589_1_motion.pdf
2,2007-12-06,1616,12562_1616_1_motion.pdf
3,2008-01-31,1635,12611_1635_1motion.pdf
4,2008-02-21,1644,12634_1644_1_interpellation.pdf
...,...,...,...
1094,2005-08-04,1362,pdoc_972_1.pdf
1095,2005-08-12,1363,pdoc_977_1.pdf
1096,2005-06-23,1364,pdoc_978_1.pdf
1097,2005-08-26,1365,pdoc_984_1.pdf


In [12]:
df_p.to_csv("KRFrames/KonvertierteDatenformate")

In [ ]:
df_p=pd.read_csv("KRFrames/KonvertierteDatenformate")

In [13]:
#Nun noch ohne.0 hinten bei der VorlageNR. Ah es ist ein Float. Kann es also zu int oder str machen
df_p["VorlageNR"]=df_p["VorlageNR"].astype(str)

In [14]:
def splitting(elem):
    elem = elem.split('.')[0].strip()
    return elem


In [16]:
df_p

,Einreichedatum,VorlageNR,Filename
0,2007-09-17,1584,12487_1584.1_motion.pdf
1,2007-09-18,1589,12493_1589_1_motion.pdf
2,2007-12-06,1616,12562_1616_1_motion.pdf
3,2008-01-31,1635,12611_1635_1motion.pdf
4,2008-02-21,1644,12634_1644_1_interpellation.pdf
...,...,...,...
1094,2005-08-04,1362,pdoc_972_1.pdf
1095,2005-08-12,1363,pdoc_977_1.pdf
1096,2005-06-23,1364,pdoc_978_1.pdf
1097,2005-08-26,1365,pdoc_984_1.pdf


In [ ]:
# Nur wenn es frisch eingelesen wird
del df_p["Unnamed: 0"] # die beiden Spalten entfernen
del df_p["Unnamed: 0.1"]

In [17]:
df_p["VorlageNR"]=df_p["VorlageNR"].apply(splitting) # So, nun sind die .0 weg. Mal schauen. Das String-Format reicht wohl

df_ps=df_p.rename(columns={"Filename": "Link Vorstoss-PDF"}) #Spalte gleich benennen wie im anderen Frame

In [18]:
df_ps.to_csv("KRFrames/df_p_Datum") # und speichern

In [31]:
df_ps=pd.read_csv("KRFrames/df_p_Datum")
df_ps

,Unnamed: 0,Einreichedatum,VorlageNR,Link Vorstoss-PDF
0,0,2007-09-17,1584.0,12487_1584.1_motion.pdf
1,1,2007-09-18,1589.0,12493_1589_1_motion.pdf
2,2,2007-12-06,1616.0,12562_1616_1_motion.pdf
3,3,2008-01-31,1635.0,12611_1635_1motion.pdf
4,4,2008-02-21,1644.0,12634_1644_1_interpellation.pdf
...,...,...,...,...
1088,1088,2005-08-04,1362.0,pdoc_972_1.pdf
1089,1089,2005-08-12,1363.0,pdoc_977_1.pdf
1090,1090,2005-06-23,1364.0,pdoc_978_1.pdf
1091,1091,2005-08-26,1365.0,pdoc_984_1.pdf


## 4.2 Ein Dataframe machen, mit den PDF-Dokumentennamen
- splitting funktion schreiben
- auf Spalte anwenden
- neu abspeichern 

In [19]:
def splittpdf(elem):
    elem = elem.split('/')[-1].strip()
    return elem

In [23]:
df_g["Link Vorstoss-PDF"]=df_g["Link Vorstoss-PDF"].apply(splittpdf)

In [24]:
df_g.to_csv("KRFrames/df_g_Datum_mit_PDF")

## 4.3 Nun zusammenfügen der beiden Daten-Dataframes anhand der PDF-Namen

In [25]:
df_g1=pd.read_csv("KRFrames/df_g_Datum_mit_PDF")
df_p1=pd.read_csv("KRFrames/df_p_Datum")

In [ ]:
df_dat_max=pd.merge(df_g1, df_p1, how="left",on= "Link Vorstoss-PDF")

In [ ]:
df_dat_max.shape

In [ ]:
df_dat_max.sort_values("Einreichedatum_x").tail(100)  #So, jetzt sind zwar beide zusammen, aber noch nicht vereint
#Jetzt noch irgendwie die beiden Einreichedaten miteinander ableichen/ergängen.

### 4.3.1 die Datenspalten zusammenbekommen.

In [ ]:
#hier füge ich sie zusammen
df_dat_zus=df_dat_max.Einreichedatum_x.combine_first(df_dat_max.Einreichedatum_y)

In [ ]:
df_dat_max['Einreichedatum'] = df_dat_zus # hier füge ich die neu enstandene "Series" dem Dataframe an

In [ ]:
df_datum=df_dat_max[["Unnamed: 0_x","VorlageNR_x", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:
df_datum.shape

In [ ]:
df_datum["Einreichedatum"].sort_values().tail(30)
                                             

In [ ]:
df_datum.to_csv("KRFrames/df_datum_ALLE")

# 5. Alle Dataframes zusammenfügen 
- Alle einlesen und auf die Unterseiten verweisen
- eines nach dem andern anhängen
- noch schön machen und kontrollieren

In [25]:
# Einlesen:
df_v=pd.read_csv("KRFrames/df_v_Hauptseiten") #Alle Vorstösse mit Infos mit Titel und Nummer der Page mit allen Geschäften
df_dat_all=pd.read_csv("KRFrames/df_datum_ALLE")# Frame mit allen Zeitangaben(Datum)(oben)
df_np=pd.read_csv("KRFrames/Frame_Namen_Partei_endversion")# Aus File "02 Namen aus Geschäftsnamen..." Alle Namen und Parteien



In [26]:
df_dat_all

,Unnamed: 0,Unnamed: 0_x,VorlageNR_x,Einreichedatum,Link Vorstoss-PDF
0,0,0,3020,2019-10-11,3020-1-16168_Racial-Profiling.pdf
1,1,1,3019,2019-10-11,3019-1-16167_Chancengleichheit.pdf
2,2,2,3018,2019-10-07,3018-1-16166_Praktikum.pdf
3,3,3,3017,2019-10-06,3017-1-16165_Frauenwahl-stimmrecht_Feier.pdf
4,4,4,3014,2019-09-26,3014-1-16159_Geschwindigkeitskontrollen-2-0.pdf
...,...,...,...,...,...
1215,1215,1215,762,2005-10-31,pdoc_1047_1.pdf
1216,1216,1216,666,2002-06-06,pdoc_55_1.pdf
1217,1217,1217,304,2003-05-27,pdoc_50_1.pdf
1218,1218,1218,2635,1995-08-07,1620


In [27]:
df_dat_all.sort_values("VorlageNR_x")

,Unnamed: 0,Unnamed: 0_x,VorlageNR_x,Einreichedatum,Link Vorstoss-PDF
1219,1219,1219,81,2007-04-10,pdoc_1705_1.pdf
1217,1217,1217,304,2003-05-27,pdoc_50_1.pdf
1216,1216,1216,666,2002-06-06,pdoc_55_1.pdf
1215,1215,1215,762,2005-10-31,pdoc_1047_1.pdf
1214,1214,1214,801,2006-08-22,pdoc_1440_1.pdf
...,...,...,...,...,...
4,4,4,3014,2019-09-26,3014-1-16159_Geschwindigkeitskontrollen-2-0.pdf
3,3,3,3017,2019-10-06,3017-1-16165_Frauenwahl-stimmrecht_Feier.pdf
2,2,2,3018,2019-10-07,3018-1-16166_Praktikum.pdf
1,1,1,3019,2019-10-11,3019-1-16167_Chancengleichheit.pdf


In [28]:
#  zuerst vorputzen
df_v1 = df_v[["VorlageNR", "Geschäft", "Zuständigkeit", "Art des Geschäfts", "Status", "url"]].copy()
df_v1.set_index("VorlageNR", inplace=True)

In [29]:
df_v1

,Geschäft,Zuständigkeit,Art des Geschäfts,Status,url
VorlageNR,,,,,
3020,Postulat von Luzian Franzini und Esther Haas b...,Sicherheitsdirektion,Postulat,hängig,/gast/geschaefte/2008
3019,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",Direktion des Innern,Motion,hängig,/gast/geschaefte/2007
3018,"Interpellation von Fabio Iten, Laura Dittli un...",Volkswirtschaftsdirektion,Interpellation,hängig,/gast/geschaefte/2006
3017,"Postulat von Tabea Zimmermann Gibson, Stéphani...",Direktion des Innern,Postulat,hängig,/gast/geschaefte/2005
3014,Interpellation der SVP-Fraktion betreffend mob...,Sicherheitsdirektion,Interpellation,hängig,/gast/geschaefte/2002
...,...,...,...,...,...
762,Motion der erweiterten Justizprüfungskommissio...,NaN,Motion,abgeschlossen,/gast/geschaefte/1331
666,Motion der Kommission Teilrevision Personalges...,NaN,Motion,abgeschlossen,/gast/geschaefte/1325
304,Motion von Manuela Weichelt betreffend HIV-Prä...,NaN,Motion,abgeschlossen,/gast/geschaefte/1309


In [30]:
df_dat_all=df_dat_all.rename(columns={"VorlageNR_x": "VorlageNR"})

In [55]:
# Nun zusammenfügen
df_test=pd.merge(df_v1, df_dat_all, how="inner",on= "VorlageNR")

In [32]:
df_alles.sort_values("Einreichedatum").tail(50)

,VorlageNR,Geschäft,Zuständigkeit,Art des Geschäfts,Status,url,Unnamed: 0,Unnamed: 0_x,Einreichedatum,Link Vorstoss-PDF
542,2220,Motion von Thomas Villiger betreffend Änderung...,Baudirektion,Motion,abgeschlossen,/gast/geschaefte/472,NaN,NaN,NaN,NaN
544,2216,Motion von Vreni Wicky und Andreas Hausheer be...,NaN,Motion,abgeschlossen,/gast/geschaefte/468,539.0,539.0,NaN,14235_2216_1_Krankenkassenpr%C3%A4mien.pdf
548,2206,Interpellation von Thomas Lötscher betreffend ...,NaN,Interpellation,abgeschlossen,/gast/geschaefte/458,543.0,543.0,NaN,14210_2206_1_Tueftellabor.pdf
549,2205,Motion der SVP-Fraktion betreffend Anpassung d...,NaN,Motion,abgeschlossen,/gast/geschaefte/457,544.0,544.0,NaN,14209_2205_1_Sozialhilfe.pdf
550,2204,Interpellation der SP-Fraktion betreffend Budg...,NaN,Interpellation,abgeschlossen,/gast/geschaefte/456,545.0,545.0,NaN,14208_2204_1_Budgetk%C3%BCrzung.pdf
552,2202,Motion der vorberatenden Kommission zum Integr...,Direktion für Bildung und Kultur,Motion,abgeschlossen,/gast/geschaefte/454,NaN,NaN,NaN,NaN
560,2187,"Motion von Leonie Winter, Thiemo Hächler und O...",Baudirektion,Motion,abgeschlossen,/gast/geschaefte/439,NaN,NaN,NaN,NaN
562,2184,Motion von André Wicki betreffend zwei Ergänzu...,Baudirektion,Motion,abgeschlossen,/gast/geschaefte/436,NaN,NaN,NaN,NaN
566,2175,Interpellation von Thomas Aeschi betreffend Üb...,NaN,Interpellation,abgeschlossen,/gast/geschaefte/427,561.0,561.0,NaN,14140_2175_1_Auftr%C3%A4ge.pdf
567,2174,Motion von Thomas Aeschi betreffend Teilrevisi...,NaN,Motion,abgeschlossen,/gast/geschaefte/426,562.0,562.0,NaN,14139_2174_1_FHG.pdf


In [ ]:
df_alles.to_csv("KRFrames/df_alles_ohne_Namen")

In [ ]:
df_alles_ganz=pd.merge(df_alles, df_np, how="left", on="VorlageNR")
df_alles_ganz.sort_values("Einreichedatum")

In [ ]:
df_tuti=df_alles_ganz[["VorlageNR","Geschäft", "Namenganz", "P", "Einreichedatum", "Link Vorstoss-PDF"]]

In [ ]:

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)
df_tuti1=df_tuti.drop_duplicates(subset=['VorlageNR', 'Namenganz'], keep=False)
# Obwohl mir nur von 30 Geschäften das Datum fehlt, explodiert dies, wenn es auf die Namen-Tabelle kommt.
df_tuti1.sort_values("Einreichedatum").tail(188)

In [ ]:
df_tuti.to_csv("KRFrames/df_tuti_Ganzes_Frame")

# Daten aus B&A mit falschen Zahlen
Eine Lösung wäre, nochmals die Links zu den PDF auslesen und diese in zwei verschiedene Listen verpacken. 
Da die Datenmigration im Jahr 2000 war, schaue ich einfach mal die Zahlen ab 2005 an. Vo dort an, werden die meisten Geschäfte tatsächlich neu sein. 
## #Andere Variante: ich lasse die B&A einfach weg. Da gibt es eh keine verlässliche Daten. Mengenmässig.
# Nun plotten

In [ ]:
df_tuti = df_tuti.set_index('Einreichedatum')

In [ ]:
df_tuti

In [ ]:
df_tuti.dtypes